In [ ]:
import numpy as np


def objective_function(params, material_use, load_capacity, P, w, t, yield_stress):
    alpha, beta, gamma = params
    return (alpha * material_use - beta * load_capacity + gamma * (P / (w * t * yield_stress)))


def firefly_algorithm(objective_function, bounds, material_use, load_capacity, P, w, t, yield_stress, num_fireflies=20, max_iter=100, alpha=0.5, beta_min=0.2, gamma=1.0):
    
    fireflies = np.random.rand(num_fireflies, len(bounds))
    for i in range(len(bounds)):
        fireflies[:, i] = bounds[i][0] + fireflies[:, i] * (bounds[i][1] - bounds[i][0])

    
    fitness = np.apply_along_axis(objective_function, 1, fireflies, material_use, load_capacity, P, w, t, yield_stress)
    best = np.min(fitness)
    best_solution = fireflies[np.argmin(fitness)]

    for iteration in range(max_iter):
        for i in range(num_fireflies):
            for j in range(num_fireflies):
                if i == j:
                    continue
                if fitness[i] > fitness[j]:
                    
                    r = np.linalg.norm(fireflies[i] - fireflies[j])
                    beta = beta_min * np.exp(-gamma * r**2)
                    fireflynew = fireflies[i].copy()
                    for k in range(len(bounds)):
                        rand_value = np.random.rand() - 0.5
                        fireflynew[k] = fireflynew[k] + beta * (fireflies[j, k] - fireflies[i, k]) + alpha * rand_value * (bounds[k][1] - bounds[k][0])
                        fireflynew[k] = np.clip(fireflynew[k], bounds[k][0], bounds[k][1])
                    

        
            newfitness = np.apply_along_axis(objective_function, 1, fireflynew[np.newaxis, :], material_use, load_capacity, P, w, t, yield_stress)
            if newfitness < fitness[i]:
                fitness[i] = newfitness
                fireflies[i] = fireflynew.copy()

        
        current_best = np.min(fitness)
        if current_best < best:
            best = current_best
            best_solution = fireflies[np.argmin(fitness)]

        
        print(f"Iteration {iteration}: Best Solution = {best_solution}, Best Fitness = {best}")

    return best_solution, best



In [ ]:

bounds = [(1, 10), (1, 10), (1, 10)]


material_use = 100  
load_capacity = 200  
P = 500  
w = 10  
t = 5  
yield_stress = 250 


best_solution, best_fitness = firefly_algorithm(objective_function, bounds, material_use, load_capacity, P, w, t, yield_stress)
print(f"Best Solution: {best_solution}")
print(f"Best Fitness: {best_fitness}")
